In [1]:
import pandas as pd
import json
import torch
import random
import torch
import tensorflow as tf
from transformers import T5ForConditionalGeneration,T5Tokenizer
from pathlib import Path
from pandas import json_normalize
from transformers import AutoTokenizer, AutoModel, AdamW, AutoModelForSeq2SeqLM
from helpers import tokenize_and_format, flat_accuracy

c:\Users\amado\anaconda3\envs\cse1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [59]:
files = {
    "train": Path("LegalSupport", "train.jsonl"),
    "dev": Path("LegalSupport", "dev.jsonl"),
    "test": Path("LegalSupport", "test.jsonl")
}
df = pd.read_json(files["train"], lines=True).explode("label")
df["citation"] = df.apply(lambda a: a["citation_a"] if a["label"] == "a" else a["citation_b"], axis=1)
df = df.drop(["citation_a", "citation_b", "label"], axis=1)
df.loc[0, "citation"]
df["signal"] = df.apply(lambda a: a["citation"]["signal"], axis=1)
df["identifier"] = df.apply(lambda a: a["citation"]["identifier"], axis=1)
df["parenthetical"] = df.apply(lambda a: a["citation"]["parenthetical"], axis=1)
df["sentence"] = df.apply(lambda a: a["citation"]["sentence"], axis=1)
df = df.drop(["citation"], axis=1)
train_df = df
# train_df = df[["context", "parenthetical"]]
train_df = train_df.rename(columns={"context": "text", "parenthetical": "label"})
len(train_df)

13862

In [61]:
train_df.drop_duplicates().iloc[1,4]

'conducting a de novo review in determining the enforceability of a foreign judgment'

In [37]:
# Question augmentation
import tensorflow as tf
model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_squad_v1')
tknizer = AutoTokenizer.from_pretrained('ramsrigouthamg/t5_squad_v1')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# model = model.to(device)
model.cuda()
dataset = list()
df = df.drop_duplicates(subset=["parenthetical"])

progress_bar = tf.keras.utils.Progbar(
    len(df),
    width=30,
    verbose=1,
    interval=0.05,
    stateful_metrics=None,
    unit_name='step'
)

for row in df.iterrows():
    text = "context: {} answer: {}".format(row[1]["context"], row[1]["parenthetical"], )
    max_len = 256
    encoding = tknizer.encode_plus(text,max_length=max_len, pad_to_max_length=False,truncation=True, return_tensors="pt")

    input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)

    outs = model.generate(input_ids=input_ids,
                          attention_mask=attention_mask,
                          early_stopping=True,
                          num_beams=5,
                          num_return_sequences=5,
                          no_repeat_ngram_size=2,
                          max_length=300)


    dec = [tknizer.decode(ids,skip_special_tokens=True) for ids in outs]

    dataset.append({"question": dec[0].replace('question:','').strip(), "answer": row[1]['parenthetical']})
    dataset.append({"question": dec[1].replace('question:','').strip(), "answer": row[1]['parenthetical']})
    progress_bar.update(row[0])

cuda:0


In [32]:
pd.DataFrame(dataset).to_csv("train_df.csv")

,question,answer,identifier,sentence
0,Witnesses are absolutely immune for the testim...,holding that while witnesses enjoy absolute im...,"25 F.3d 83, 83","Malachowski, 787 F.2d at 712; see also Dory, 2..."
1,Witnesses are absolutely immune for the testim...,holding that while witnesses enjoy absolute im...,"25 F.3d 83, 83","Malachowski, 787 F.2d at 712; see also Dory, 2..."
2,We exercise de novo review over a district cou...,conducting a de novo review in determining the...,"489 F.2d 661, 662-63","See Barrows v. Barrows, 489 F.2d 661, 662-63 (..."
3,What do we exercise de novo review over a dist...,conducting a de novo review in determining the...,"489 F.2d 661, 662-63","See Barrows v. Barrows, 489 F.2d 661, 662-63 (..."
4,We do not reach the merits of the issues raise...,recognizing federal court's duty to determine ...,"859 F.2d 842, 844","Auto. Ass’n, 859 F.2d 842, 844 (10th Cir. 1988..."
5,We do not reach the merits of the issues raise...,recognizing federal court's duty to determine ...,"859 F.2d 842, 844","Auto. Ass’n, 859 F.2d 842, 844 (10th Cir. 1988..."
6,"What is required to ""cause"" the presentation o...",doctor caused false claims to be submitted to ...,"261 F.3d 827, 827","Mackby, 261 F.3d at 827 (doctor caused false c..."
7,"What is required to ""cause"" the presentation o...",doctor caused false claims to be submitted to ...,"261 F.3d 827, 827","Mackby, 261 F.3d at 827 (doctor caused false c..."
8,Some degree of participation in the claims pro...,defendant liable where he delegated to his wif...,"111 F.3d 943, 943","Mackby, 261 F.3d at 827 (doctor caused false c..."
9,Some degree of participation in the claims pro...,defendant liable where he delegated to his wif...,"111 F.3d 943, 943","Mackby, 261 F.3d at 827 (doctor caused false c..."
